# Generate word2vec model - include all new topic words



In [1]:
from gensim.models import Word2Vec, KeyedVectors
import gensim
import time
import numpy as np
from tqdm import tqdm
from pprint import pprint
import pickle
import spacy
import re
import operator
import spacy_udpipe

In [2]:
spacy_udpipe.download('lv')
nlp = spacy_udpipe.load('lv')

Downloaded pre-trained UDPipe model for 'lv' language


In [3]:
# set variables 
LANG = 'lv'
MAX_WORDS = 500000  # maximum processed words to small model 

MODEL_DIR = '/mnt/data/data/models-fasttext/'
TOPICS_DIR = '/home/dzon/kajo/topics/topic_lists-based_on_en-US/'

model_filename = MODEL_DIR + 'cc.'+ LANG +'.300-lemma_500k_min2.vec.bin'

excluded_model_name = MODEL_DIR + 'cc.'+ LANG +'.300-lemma_500k_min2_excluded.vec.bin'

model_orig_filename = MODEL_DIR + 'cc.'+ LANG +'.300.vec.bin'
model_orig_filename_vec = MODEL_DIR + 'cc.'+ LANG +'.300.vec.gz'

pickled_words_lemmas = MODEL_DIR + LANG + '_ft_all_words_lemmas.pickle'

lemma_list_file = TOPICS_DIR + 'topics_lemma_'+ LANG +'.lst'
nolemma_list_file = TOPICS_DIR + 'topics_'+ LANG +'.lst'

excluded_filename = TOPICS_DIR + 'topics_' + LANG + '-not_in_model.lst'

lemma_list_file_excluded = TOPICS_DIR + 'topics_lemma_excluded_' + LANG + '.lst'
nolemma_list_file_excluded = TOPICS_DIR + 'topics_excluded_' + LANG + '.lst'


In [4]:
# load the big model
model_big = KeyedVectors.load_word2vec_format(model_orig_filename, binary=True)

In [5]:
# process all words in fasttext model  takes ~7min for LV
all_words_lemmas = []

lemmas = {}
word_lemma = {}
words = model_big.index2word
for word in tqdm(words):
    # skip "words" which are not regular words
    if not re.fullmatch("([^\d\W])+", word):
        continue
    doc = nlp(word)
    lemma = doc[0].lemma_.lower()
    all_words_lemmas.append((word, lemma))
print("Number of words in big model: ", len(all_words_lemmas))

#save pickled dump 
with open(pickled_words_lemmas, 'wb') as f:
    pickle.dump(all_words_lemmas, f)

100%|██████████| 1641469/1641469 [06:05<00:00, 4492.79it/s]


Number of words in big model:  1314118


In [6]:
#load pickled lemma list
with open(pickled_words_lemmas, 'rb') as f:
    all_words_lemmas = pickle.load(f)
pprint(all_words_lemmas[:50])

[('un', 'un'),
 ('ir', 'būt'),
 ('ar', 'ar'),
 ('par', 'par'),
 ('no', 'no'),
 ('arī', 'arī'),
 ('uz', 'uz'),
 ('kas', 'kas'),
 ('vai', 'vai'),
 ('kā', 'kā'),
 ('ka', 'ka'),
 ('gada', 'gads'),
 ('gadā', 'gads'),
 ('bija', 'būt'),
 ('_', '_'),
 ('to', 'tā'),
 ('bet', 'bet'),
 ('Latvijas', 'latvija'),
 ('līdz', 'līdz'),
 ('pēc', 'pēc'),
 ('var', 'varēt'),
 ('lai', 'lai'),
 ('tika', 'tikt'),
 ('nav', 'būt'),
 ('tā', 'tā'),
 ('tiek', 'tikt'),
 ('tikai', 'tikai'),
 ('gan', 'gan'),
 ('tas', 'tas'),
 ('laikā', 'laiks'),
 ('jau', 'jau'),
 ('pie', 'pie'),
 ('ko', 'kas'),
 ('valsts', 'valsts'),
 ('jo', 'jo'),
 ('savu', 'savs'),
 ('ja', 'ja'),
 ('Rīgas', 'rīga'),
 ('tās', 'tā'),
 ('šo', 'šī'),
 ('kad', 'kad'),
 ('vairāk', 'daudz'),
 ('tad', 'tad'),
 ('Latvijā', 'latvija'),
 ('Lai', 'lai'),
 ('kur', 'kur'),
 ('ļoti', 'ļoti'),
 ('vietā', 'vieta'),
 ('darba', 'darbs'),
 ('viņš', 'viņš')]


In [7]:
# generate helpful dictionaries/lists
big_lemmas_set = set([l for (_,l) in all_words_lemmas])
print(len(big_lemmas_set))

big_words_set = set([w for (w,_) in all_words_lemmas])
print(len(big_words_set))

lemmas = {}
words_lower_orig = {}
big_lemma_word_dic = {}
big_word_lemma_dic = {}
for (w,l) in all_words_lemmas:
    big_word_lemma_dic[w] = l
    
    #dictionary of lowercase word -> all original words
    if w.lower() in words_lower_orig:
        words_lower_orig[w.lower()].append(w)
    else:
        words_lower_orig[w.lower()] = [w]
        
    if l in big_lemma_word_dic:
        big_lemma_word_dic[l].append(w)
        lemmas[l] += 1
    else:
        big_lemma_word_dic[l] = [w]
        lemmas[l] = 1
        
# load all the words from big model
words = model_big.wv.index2word
words_lower = [w.lower() for w in words]

#words_lemma = model_lemma_big.index2word       

747102
1314118


/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [8]:
# sort the lemmas from most frequent
sorted_lemmas = sorted(lemmas.items(), key=operator.itemgetter(1), reverse=True)
# trying first words
sorted_lemmas[:20]

[('būt', 208),
 ('redzēt', 182),
 ('izmantot', 164),
 ('zināt', 147),
 ('gaidīt', 142),
 ('atklāt', 141),
 ('liels', 137),
 ('gribēt', 137),
 ('domāt', 133),
 ('aizsargāt', 132),
 ('jauns', 131),
 ('labs', 129),
 ('atbalstīt', 128),
 ('radīt', 128),
 ('runāt', 127),
 ('atjaunot', 127),
 ('iegūt', 126),
 ('varēt', 124),
 ('dzīvot', 123),
 ('veikt', 123)]

In [9]:
# create vectors for the small models - average vectors for lemmas
# process only first MAX_WORDS in the big model
tmp_words = [None] * len(model_big.wv.index2word)
tmp_vecs = [None] * len(model_big.wv.index2word) 
for i,(word,lemma) in enumerate(all_words_lemmas[:MAX_WORDS]):
    lemma_words_list = big_lemma_word_dic[lemma]
    # add only lemmas made from at least 2 words
    if len(lemma_words_list)<2:
        continue
    tmp_vectors = []
    tmp_indexes = []
    error_with_word = False
    for word2 in lemma_words_list:
        try:
            tmp_vectors.append(model_big[word2])  
            tmp_indexes.append(model_big.vocab[word2].index)
        except:
            print(i, "error with ", word, " lemma ", lemma, "words", lemma_words_list)
            error_with_word = True
            continue
    if not error_with_word:
        try:
            tmp_words[min(tmp_indexes)] = lemma
            tmp_vecs[min(tmp_indexes)] = list(np.average(tmp_vectors, axis=0))
        except:
            print(i, "error with lemma ", lemma, " - words", lemma_words_list, " indexes: ", tmp_indexes)
            # note - we are processing first MAX_WORDS from the model after the filter out of the "non-words"
            # so some of the last words in that range may refer to words on bigger index than MAX_WORDS

# filter the lists (some positions remained None)
final_words = [i for i in tmp_words if i] 
final_vecs = [i for i in tmp_vecs if i != None] 

print(len(final_vecs), len(final_words))
len(final_vecs[0])  

del tmp_vecs
del tmp_words

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


122615 122615


In [10]:
# create new gensim word2vec model with the vectors:
new_vector_space = gensim.models.keyedvectors.Word2VecKeyedVectors(300)
new_vector_space.add(final_words,final_vecs)

In [13]:
# test the model
pprint(new_vector_space.most_similar('tēvs'))
pprint(new_vector_space.most_similar_cosmul(positive=['sieviete', 'karalis'], negative=['cilvēks']))

[('dēls', 0.7674477100372314),
 ('māte', 0.7387697100639343),
 ('tēvutēvs', 0.6727820634841919),
 ('tēvabrālis', 0.6689823865890503),
 ('meita', 0.6421108245849609),
 ('patēvs', 0.6383793950080872),
 ('audžutēvs', 0.6289832592010498),
 ('vīratēvs', 0.6251531839370728),
 ('vectēvs', 0.623360276222229),
 ('brālis', 0.6206009387969971)]
[('karaliene', 0.8439878821372986),
 ('šorta', 0.8204429149627686),
 ('karal', 0.8112600445747375),
 ('karalienes', 0.8077372908592224),
 ('birgita', 0.8050732016563416),
 ('karals', 0.8045182824134827),
 ('princeses', 0.8026629686355591),
 ('karalaucis', 0.7986181378364563),
 ('princese', 0.797778844833374),
 ('dāma', 0.7976422905921936)]


In [14]:
# save the model
new_vector_space.save_word2vec_format(model_filename, binary=True)

In [15]:
#model = KeyedVectors.load_word2vec_format(model_filename, binary=True)
model = new_vector_space
words_model = model.wv.index2word

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
# load topic listst
with open(nolemma_list_file) as fp:
    topics_nolemma = fp.readlines() 
print(len(topics_nolemma))
    
# load the lemma list
# with open(lemma_list_file) as fp:
#    topics = fp.readlines()
# print(len(topics))

topics = []
# generate the lemma list
for topic in tqdm(topics_nolemma):
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []
        
    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    if len(lemmas) == 0:
        print('lemmas = 0 ', topics_nolemma)
    topics.append("_".join(lemmas))
    
print(len(topics))    
    
#write the lemma topic list
with open(lemma_list_file, 'wt', encoding='utf8') as f:
    for topic_lemma in topics:
        f.write(topic_lemma + "\n")    

  2%|▏         | 85/5308 [00:00<00:06, 845.38it/s]

5308


100%|██████████| 5308/5308 [00:04<00:00, 1171.17it/s]

5308


In [17]:
print(len(topics),len(topics_nolemma))  

5308 5308


In [18]:
#try if the topic-lemma words exist in the models  

words_to_manual_add = {}

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic = topic.strip()
    topic_words = topic.split('_')
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    for word, word_nolemma in zip(topic_words, nolemmas):
        #print(word)
        found = 0
        if word in words_model:
            found = 1
            #print("l1")
        elif word in big_lemmas_set:
            found = 1
            print("l2", word)
        elif word in words_lower:
            found = 1
            print("l3", word)
        # nolemma words are in PL case probably not needed and all lemmas exist in lemma list
        elif word_nolemma in words_model:
            found = 1
            print("w1", word)
        elif word_nolemma in big_lemmas_set:
            found = 1
            print("w2", word)
        elif word_nolemma in words_lower:
            found = 1
            print("w3", word)

        if found == 0:
            print(i, 'needs to be added manually: ', word, " - ", word_nolemma, " - ", topic_nolemma.strip())
            if word not in words_to_manual_add:
                words_to_manual_add[word] = topic_nolemma.strip()
                    
print(len(words_to_manual_add))          

l2 sākumizglītība
l2 pārstrādes
112 needs to be added manually:  sasāļošana  -  sasāļošana  -  augsnes sasāļošana
l2 tiesība
166 needs to be added manually:  kredītpalīdzība  -  kredītpalīdzība  -  kredītpalīdzība
196 needs to be added manually:  atsārņošana  -  atsārņošana  -  augsnes atsārņošana
w1 tēraudīt
291 needs to be added manually:  mēbeļrūpniecība  -  mēbeļrūpniecība  -  mēbeļrūpniecība
l2 pārpurvojuties
323 needs to be added manually:  ietekmējums  -  ietekmējums  -  vides ietekmējums
349 needs to be added manually:  klimatefekts  -  klimatefekts  -  klimatefekts
398 needs to be added manually:  komunālsaimniecība  -  komunālsaimniecības  -  komunālsaimniecības objekts
l2 agrotehnoloģija
l3 augsnē
l3 sekas
l2 gadagājums
l3 sistēmas
l2 energotehnoloģija
l2 mazskarts
l2 iesāļš
l2 sārmainīt
w1 urbānāties
692 needs to be added manually:  korozijaktivitāte  -  korozijaktivitāte  -  ūdens korozijaktivitāte
l3 izstrādājuma
l2 būvapkalums
w3 antarktīdēties
w3 divvs
l2 energoekonomik

l2 duļķainums
3554 needs to be added manually:  urbiekārta  -  urbiekārtas  -  urbiekārtas montāža
3555 needs to be added manually:  aisījums  -  aisījumi  -  polimēra piem aisījumi
l2 pretsēnes
l2 disperģēšana
l2 ilgtermiņš
l3 mājsaimniecības
3607 needs to be added manually:  aizsargiekārst  -  aizsargiekārta  -  aizsargiekārta
3646 needs to be added manually:  imisija  -  imisiju  -  imisiju ierobežojums
3657 needs to be added manually:  gāzdzinējs  -  gāzdzinējs  -  gāzdzinējs
3666 needs to be added manually:  vitrifikācija  -  vitrifikācija  -  vitrifikācija
l2 ilgtermiņš
l3 attiecības
w3 izpildvarties
l2 halogenēt
l2 citotoksiskums
3718 needs to be added manually:  podziņelements  -  podziņelements  -  podziņelements baterija
l2 absorbēšanās
3754 needs to be added manually:  imisija  -  imisiju  -  imisiju radītais kaitējums
3767 needs to be added manually:  dehlorēšana  -  dehlorēšana  -  dehlorēšana
3773 needs to be added manually:  apgredzenošana  -  apgredzenošana  -  apgredze

l2 līdzsadedzināšana
l2 dozējums
4952 needs to be added manually:  pentahlorfenols  -  pentahlorfenols  -  pentahlorfenols
4956 needs to be added manually:  onhocerikoze  -  onhocerikoze  -  onhocerikoze upju aklums
l2 iedīglt
l2 etilēt
l2 kanceroģenēze
l2 halogenēt
4985 needs to be added manually:  hlorfluorūdeņradis  -  hlorfluorūdeņradis  -  daļēji halogenēts hlorfluorūdeņradis
l2 pārtvaicēšana
l2 ģeotehnika
l2 izzudties
5005 needs to be added manually:  kancerogēniskums  -  kancerogēniskuma  -  kancerogēniskuma tests
l2 horoloģija
l2 teratoģenēze
5018 needs to be added manually:  desulfurizēšana  -  desulfurizēšana  -  desulfurizēšana
l2 sārmainums
l2 spazmatisks
5027 needs to be added manually:  hlorofenols  -  hlorofenols  -  hlorofenols
5036 needs to be added manually:  pierādāmība  -  pierādāmība  -  pierādāmība rādītspēja
5036 needs to be added manually:  rādītspēt  -  rādītspēja  -  pierādāmība rādītspēja
l2 benzopirēns
l2 enterovīruss
5043 needs to be added manually:  ekoped

In [19]:
# for manual correcting of words
pprint(words_to_manual_add)

{'aerobioloģija': 'aerobioloģija',
 'aerosolveida': 'atmosfēras aerosolveida piesārņojums',
 'agrometeoroloģija': 'agrometeoroloģija',
 'agromežsaimniecība': 'agromežsaimniecība',
 'agrosistēma': 'agrosistēma',
 'aisījums': 'polimēra piem aisījumi',
 'aizsargiekārst': 'aizsargiekārta',
 'algicīds': 'algicīds',
 'alkils': 'alkila savienojums',
 'anelīds': 'anelīds',
 'antiseismisks': 'antiseismiskie noteikumi',
 'antropisks': 'antropiskā darbība',
 'apgredzenošana': 'apgredzenošana savvaļa',
 'atkritumeļļa': 'atkritumeļļa',
 'atlapošana': 'atlapošana defoliācija',
 'atmosfērzinātne': 'atmosfērzinātne',
 'atrazīns': 'atrazīns',
 'atsārņošana': 'augsnes atsārņošana',
 'atsērošana': 'dūmvadu gāzes atsērošana',
 'bentāls': 'bentālā ekosistēma',
 'biofiltrēšana': 'biofiltrēšana',
 'biokoncentrācija': 'biokoncentrācijas faktors',
 'biostimulants': 'augu biostimulants',
 'bioģeoķīmisks': 'bioģeoķīmiskais cikls',
 'birokratizēšanās': 'birokratizēšanās',
 'bradiseisms': 'bradiseisms',
 'briofīt'

In [20]:
# the lemmas not in model were manually checked if they exist in other models in some variation
word = "Etelänapamanner"
print(word)
try:
    model[word]
    print('model')
    pprint(model.most_similar(word))
except:
    pass

if word in big_lemmas_set:
    print('model_lemma_big', big_lemma_word_dic[word])

try:
    model_big[word]
    print('model big')
    pprint(model_big.most_similar(word))
except:
    pass

# if the original word with uppercase
if word in words_lower_orig:
    print('model big LOWER', words_lower_orig[word])
    pprint(model_big.most_similar(words_lower_orig[word]))
    

Etelänapamanner


In [21]:
# for cases the lemmas aren't correct or don't exist in the dic
# manually made dictionary of "lemma" : "word which exists in some model" or "word1;word2" for the average
# e.g. 'mikroekosystém': 'mikro;ekosystém', 'bahný': 'bahno'
new_words_manually_checked = {}
    
len(new_words_manually_checked)

0

In [22]:
# function which finds the word in all the available models and returns it's vector (average vector)
def get_vector(word):
    # word is in small model
    try:
        return model[word]
    except:
        pass
    
    # word is in lemmas from big model - return average if the lemma was result of more words
    if word in big_lemmas_set:
        lemma_words_list = big_lemma_word_dic[word]
        tmp_vectors = []
        for word2 in lemma_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

    try:
        return model_big[word]
    except:
        pass
    
    # if the original word with uppercase
    if word in words_lower_orig:
        orig_words_list = words_lower_orig[word]
        tmp_vectors = []
        for word2 in orig_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

        #return model_big[words_lower_orig[word]]
    
    return []

In [23]:
# check if the word vectors are the same
#get_vector('Antarktyki') == get_vector('antarktyki')
get_vector('διβενζοφουράνιο')

[]

In [24]:
# ading words from manually checked list
new_vectors = []
new_words = []
for i, word in enumerate(new_words_manually_checked.keys()):
    
    tmp_vectors = []
    print(i, word)
    for tmp_word in new_words_manually_checked[word].split(';'):
        vector = get_vector(tmp_word)
        if vector != []:
            tmp_vectors.append(vector)
        else:
            print("XXX - some ERROR, word doesn't have vector, cannot add: ", tmp_word)
    if word not in new_words:
        new_vectors.append(np.average(tmp_vectors, axis=0))
        new_words.append(word)
print(len(new_vectors),len(new_words))        

0 0


In [25]:
# add new words to the model
if len(new_words) > 0:
    model.add(new_words,new_vectors)

In [26]:
print(len(topics),len(topics_nolemma))  

5308 5308


In [27]:
# 2nd run - checking if there are still unavailable words in topics
excluded_topics = []
#try if the topic-lemma words exist in the model   

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma_orig = topic_nolemma.strip()
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic_words = topic.split('_')
    
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
        
    found = 1
    for word, word_nolemma in zip(topic_words, nolemmas):
    #for word in topic_words:
        #print(word)
       
        vector = get_vector(word)
        if vector == []:
            print('nonexistend word: ', word, " - ", topic_nolemma)
            vector2 = get_vector(word_nolemma)
            if vector2 == []:
                found = 0
                print('nonexistend word: ', word_nolemma, " - ", topic_nolemma)
            else:
                print("Exist nolemma")

    if found == 0:
        print('!!! topic needs to be excluded: ', topic_nolemma_orig)
        if topic_nolemma.strip() not in excluded_topics:
            excluded_topics.append(topic_nolemma_orig)
                    
print(len(excluded_topics)) 

nonexistend word:  sasāļošana  -  augsnes sasāļošana 
nonexistend word:  sasāļošana  -  augsnes sasāļošana 
!!! topic needs to be excluded:  augsnes sasāļošana
nonexistend word:  kredītpalīdzība  -  kredītpalīdzība 
nonexistend word:  kredītpalīdzība  -  kredītpalīdzība 
!!! topic needs to be excluded:  kredītpalīdzība
nonexistend word:  atsārņošana  -  augsnes atsārņošana 
nonexistend word:  atsārņošana  -  augsnes atsārņošana 
!!! topic needs to be excluded:  augsnes atsārņošana
nonexistend word:  tēraudīt  -  tērauda rūpniecība 
Exist nolemma


/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


nonexistend word:  mēbeļrūpniecība  -  mēbeļrūpniecība 
nonexistend word:  mēbeļrūpniecība  -  mēbeļrūpniecība 
!!! topic needs to be excluded:  mēbeļrūpniecība
nonexistend word:  ietekmējums  -  vides ietekmējums 
nonexistend word:  ietekmējums  -  vides ietekmējums 
!!! topic needs to be excluded:  vides ietekmējums
nonexistend word:  klimatefekts  -  klimatefekts 
nonexistend word:  klimatefekts  -  klimatefekts 
!!! topic needs to be excluded:  klimatefekts
nonexistend word:  komunālsaimniecība  -  komunālsaimniecības objekts 
nonexistend word:  komunālsaimniecības  -  komunālsaimniecības objekts 
!!! topic needs to be excluded:  komunālsaimniecības objekts
nonexistend word:  urbānāties  -  urbānās struktūras 
Exist nolemma
nonexistend word:  korozijaktivitāte  -  ūdens korozijaktivitāte 
nonexistend word:  korozijaktivitāte  -  ūdens korozijaktivitāte 
!!! topic needs to be excluded:  ūdens korozijaktivitāte
nonexistend word:  antarktīdēties  -  Antarktīdas reģions 
Exist nolemma


nonexistend word:  pašpārraudzība  -  pašpārraudzība 
nonexistend word:  pašpārraudzība  -  pašpārraudzība 
!!! topic needs to be excluded:  pašpārraudzība
nonexistend word:  impulsveidīt  -  impulsveida troksnis 
Exist nolemma
nonexistend word:  tālpārvadājums  -  tālpārvadājumi 
nonexistend word:  tālpārvadājumi  -  tālpārvadājumi 
!!! topic needs to be excluded:  tālpārvadājumi
nonexistend word:  civilceltniecība  -  būvniecība civilceltniecība 
nonexistend word:  civilceltniecība  -  būvniecība civilceltniecība 
!!! topic needs to be excluded:  būvniecība; civilceltniecība
nonexistend word:  flokulācija  -  flokulācija 
nonexistend word:  flokulācija  -  flokulācija 
!!! topic needs to be excluded:  flokulācija
nonexistend word:  nosēdtvertne  -  nosēdtvertne 
nonexistend word:  nosēdtvertne  -  nosēdtvertne 
!!! topic needs to be excluded:  nosēdtvertne
nonexistend word:  dambt  -  dambja stiprinājums 
Exist nolemma
nonexistend word:  fitotoksicitāte  -  fitotoksicitāte 
nonexiste

nonexistend word:  iekārsties  -  atkritumu izmantošana apkures iekārtās 
Exist nolemma
nonexistend word:  piesārņojošt  -  nepiesārņojošs enerģijas avots 
nonexistend word:  nepiesārņojošs  -  nepiesārņojošs enerģijas avots 
!!! topic needs to be excluded:  nepiesārņojošs enerģijas avots
nonexistend word:  imisija  -  piesārņotāja imisija 
nonexistend word:  imisija  -  piesārņotāja imisija 
!!! topic needs to be excluded:  piesārņotāja imisija
nonexistend word:  pildizgāztuve  -  pildizgāztuves segums 
nonexistend word:  pildizgāztuves  -  pildizgāztuves segums 
!!! topic needs to be excluded:  pildizgāztuves segums
nonexistend word:  fitosocioloģija  -  fitosocioloģija 
nonexistend word:  fitosocioloģija  -  fitosocioloģija 
!!! topic needs to be excluded:  fitosocioloģija
nonexistend word:  paleoekoloģija  -  paleoekoloģija 
nonexistend word:  paleoekoloģija  -  paleoekoloģija 
!!! topic needs to be excluded:  paleoekoloģija
nonexistend word:  septiktvertne  -  septiktvertne 
nonex

nonexistend word:  posidonija  -  posidonija 
!!! topic needs to be excluded:  posidonija
nonexistend word:  ekomuzejs  -  ekomuzejs 
nonexistend word:  ekomuzejs  -  ekomuzejs 
!!! topic needs to be excluded:  ekomuzejs
nonexistend word:  agrosistēma  -  agrosistēma 
nonexistend word:  agrosistēma  -  agrosistēma 
!!! topic needs to be excluded:  agrosistēma
nonexistend word:  piralīns  -  piralīns 
nonexistend word:  piralīns  -  piralīns 
!!! topic needs to be excluded:  piralīns
nonexistend word:  stāvūdens  -  dīķu sistēma stāvūdeņu attīrīšanai 
nonexistend word:  stāvūdeņu  -  dīķu sistēma stāvūdeņu attīrīšanai 
!!! topic needs to be excluded:  dīķu sistēma stāvūdeņu attīrīšanai
nonexistend word:  biofiltrēšana  -  biofiltrēšana 
nonexistend word:  biofiltrēšana  -  biofiltrēšana 
!!! topic needs to be excluded:  biofiltrēšana
nonexistend word:  metanizācija  -  metanizācija 
nonexistend word:  metanizācija  -  metanizācija 
!!! topic needs to be excluded:  metanizācija
nonexiste

In [28]:
# save topics containing words which are not present neither model - to be excluded from the topic list
print('excluded topics: ', len(excluded_topics))
pprint(excluded_topics)
with open(excluded_filename, 'wt', encoding='utf8') as f:
    for topic in excluded_topics:
        f.write(topic + "\n")

excluded topics:  182
['augsnes sasāļošana',
 'kredītpalīdzība',
 'augsnes atsārņošana',
 'mēbeļrūpniecība',
 'vides ietekmējums',
 'klimatefekts',
 'komunālsaimniecības objekts',
 'ūdens korozijaktivitāte',
 'lauksaimniecības hidrotehnoloģijas',
 'dzesētājūdens; dzesēšanas ūdens',
 'antiseismiskie noteikumi',
 'darbalgas sistēma',
 'netīrais ūdens nosēdtilpnē',
 'fizioķīmisks process',
 'vides ietekmējuma novērtējums',
 'kultūrsociāla grupa',
 'bentālā ekosistēma',
 'transports tālpārvadājumiem',
 'energosadales sistēma',
 'ietvērums (kodolnozare)',
 'vides veselības ietekmējuma novērtējums',
 'imisiju kontroles likums',
 'vides pārskats (ekoaudits)',
 'burbuļpolitika (emisijas kvotu tirdzniecība)',
 'pamatgrunts drenāža',
 'agromežsaimniecība',
 'pamatzeme, pamatgrunts',
 'atlapošana; defoliācija',
 'mikromaisījums',
 'uz ceļa kaisāmā sāls',
 'maiņveida kultivēšana',
 'petroķīmisks',
 'mulčējums',
 'marikultūra',
 'tekstilnozarē izmantojams augs',
 'jaunaudze ar standartkokiem',
 'at

In [29]:
# create new lists which don't containt topics with non-existent words

# load list with excluded topics
with open(excluded_filename, 'rt', encoding='utf8') as fp:
    tmp_topics = fp.readlines()
excluded_set = set()
for topic in tmp_topics:
    excluded_set.add(topic.strip())

with open(nolemma_list_file, 'rt', encoding='utf8') as fp:
        topics_file = fp.readlines()

topics_final = []
topics_lemma = []

excluded = 0
for topic in topics_file:
    topic_orig = topic.strip()
    if topic_orig in excluded_set:
        print("=== EXCLUDE ", topic_orig)
        excluded += 1
        continue
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []

    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    topics_lemma.append("_".join(lemmas))
    topics_final.append(topic_orig)

    print(topic_orig, ' --- ', "_".join(lemmas))

print('Excluded: ', excluded)
print('Final topics: ', len(topics_lemma), len(topics_final))
    
with open(lemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_lemma in topics_lemma:
        f.write(topic_lemma + "\n")
with open(nolemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_final in topics_final:
        f.write(topic_final + "\n")  

vietējā attīstība  ---  vietējs_attīstība
lauku attīstība  ---  lauks_attīstība
ūdens resursu apsaimniekošana  ---  ūdens_resursi_apsaimniekošana
izglītības līmenis  ---  izglītība_līmenis
vispārējā izglītība  ---  vispārējs_izglītība
banka (zeme)  ---  banka_zeme
Kopienas tiesības  ---  kopiena_tiesības
kultūras mantojums  ---  kultūra_mantojums
lauksaimniecības zeme  ---  lauksaimniecības_zeme
lauksaimniecības ainava  ---  lauksaimniecības_ainava
lauksaimnieciskā ražošana  ---  lauksaimniecisks_ražošana
ekonomiskā attīstība  ---  ekonomisks_attīstība
barības ķēde  ---  barība_ķēde
pārtikas rūpniecība  ---  pārtika_rūpniecība
pārtikas kvalitāte  ---  pārtika_kvalitāte
globālā sasilšana  ---  globāls_sasilšana
siltumnīcas efekts  ---  siltumnīca_efekts
rūpnieciskā attīstība  ---  rūpniecisks_attīstība
darba tirgus  ---  darbs_tirgus
pieeja zemei  ---  pieeja_zeme
zemes gabalu apvienošana  ---  zeme_gabals_apvienošana
zemes plānošana  ---  zeme_plānošana
ainavu arhitektūra  ---  ainava_

politika  ---  politika
meža izmantošana  ---  mežs_izmantošana
zemes atjaunošana  ---  zeme_atjaunošana
transporta sistēma  ---  transports_sistēma
augsnes sāļums  ---  augsne_sāļums
ūdens sāļums  ---  ūdens_sāļums
attīstības plānošana  ---  attīstība_plānošana
transporta plānošana  ---  transports_plānošana
lauksaimniecības aprīkojums  ---  lauksaimniecības_aprīkojums
lauksaimniecības dzīvnieks  ---  lauksaimniecības_dzīvnieks
tiesību akti par rūpniecību  ---  tiesības_akts_rūpniecība
finansiālā palīdzība  ---  finansiāls_palīdzība
vides kvalitāte  ---  vide_kvalitāte
augsnes mitrums  ---  augsne_mitrums
lauksaimniecības vadība  ---  lauksaimniecības_vadība
piena lopu ferma; piena ražošanas saimniecība  ---  piens_lops_ferma_piens_ražošana_saimniecība
elektrotehnikas rūpniecība  ---  elektrotehnika_rūpniecība
pārtikas konservēšana  ---  pārtika_konservēšana
sateces baseins  ---  satece_baseins
sociālā aprūpe  ---  sociāls_aprūpe
vide  ---  vide
piekrastes erozija  ---  piekraste_eroz

bioloģiskā daudzveidība  ---  bioloģisks_daudzveidība
finansiāla kompensācija  ---  finansiāls_kompensācija
pilsētas sanitārijas pasākumi  ---  pilsēta_sanitārija_pasākums
dzesēšanas rūpniecība  ---  dzesēšana_rūpniecība
ūdens atkārtota izmantošana  ---  ūdens_atkārtot_izmantošana
bruņotie spēki  ---  bruņots_spēks
lauksaimniecības kaitēkļi  ---  lauksaimniecības_kaitēklis
dabiska (mazskarta) teritorija  ---  dabisks_mazskarts_teritorija
uzņēmējdarbība  ---  uzņēmējdarbībs
sociālie apstākļi  ---  sociāls_apstāklis
cukura rūpniecība  ---  cukurs_rūpniecība
jūras inženierija  ---  jūra_inženierija
administratīvās tiesības  ---  administratīvs_tiesības
attīstības paraugs  ---  attīstība_paraugs
augsnes bioloģija  ---  augsne_bioloģija
iedzīvotāju kustība  ---  iedzīvotājs_kustība
ērti lietojama pārtika  ---  ērti_lietot_pārtika
māksla  ---  māksla
ekonomiskā ģeogrāfija  ---  ekonomisks_ģeogrāfija
inovācija; jauninājums  ---  inovācija_jauninājums
augsnes stabilitāte  ---  augsne_stabilitā

ūdens dzirnavas  ---  ūdens_dzirnavas
ūdenstilpe, kas atrodas valsts īpašumā  ---  ūdenstilpe_kas_atrasties_valsts_īpašums
izmantotais ūdens  ---  izmantot_ūdens
ūdens masa  ---  ūdens_mass
aka  ---  aka
vides atskaite  ---  vide_atskaite
ūdens apdraudēšana  ---  ūdens_apdraudēšana
vides kontrole  ---  vide_kontrole
iesaiņojuma rūpniecība  ---  iesaiņojums_rūpniecība
gruntsūdeņu kvalitāte  ---  gruntsūdens_kvalitāte
ūdens plūsmas zemākais līmenis  ---  ūdens_plūsma_zems_līmenis
enerģijas saglabāšana  ---  enerģija_saglabāšana
kvalitātes mērķa rādītājs  ---  kvalitāte_mērķis_rādītājs
ražošanas tehnoloģija  ---  ražošana_tehnoloģija
ārpolitika  ---  ārpolitika
Eiropas Kopienas  ---  eiropa_kopiena
vides ekonomika  ---  vide_ekonomika
paplašināšanas politika  ---  paplašināsšana_politika
ekspertu sistēma  ---  eksperts_sistēma
sārmaina zeme  ---  sārmainīt_zeme
trofiskais līmenis  ---  trofisks_līmenis
Vidusjūras zona  ---  vidusjūra_zona
ģeomorfoloģiskais process  ---  ģeomorfoloģisks_pr

arodveselība  ---  arodveselība
piesārņotāja līmenis  ---  piesārņotāt_līmenis
nervu sistēma  ---  nervs_sistēma
ārējā tirdzniecība  ---  ārējs_tirdzniecība
resursu rezerve  ---  resursi_rezerve
spiestuve  ---  spiestuve
augšpalāta  ---  augšpalāts
dzīvnieku sugas  ---  dzīvnieks_sugas
ceļa drošība  ---  ceļš_drošība
uzbūvēta struktūra  ---  uzbūvēt_struktūra
vides jēdziens  ---  vide_jēdziens
putnu aizsardzība  ---  putns_aizsardzība
ģeoloģiskais process  ---  ģeoloģisks_process
ķīmiskā struktūra  ---  ķīmisks_struktūra
patoloģisks efekts  ---  patoloģisks_efekts
=== EXCLUDE  fizioķīmisks process
siltumnīcu saimniecība  ---  siltumnīca_saimniecība
atkritumu termiskā apstrāde  ---  atkritumi_termisks_apstrāde
koeficients; rādītājs  ---  koeficients_rādītājs
būvlaukums  ---  būvlaukums
Kopienas ekomarķējums  ---  kopiena_ekomarķējums
plānotā urbānā attīstība  ---  plānot_urbāns_attīstība
pārošanai derīgs putns  ---  pārošana_derīgs_putns
mīlestība pret dabu  ---  mīlestība_daba
mērenās 

...

uzvedība miera periodā  ---  uzvedība_miers_periods
kodolreakcija  ---  kodolreakcija
vējdzirnavas  ---  vējdzirnavas
kastanis  ---  kastanis
šķīdība  ---  šķīdība
reģistrācijas process  ---  reģistrācija_process
sanitārā izgāztuve  ---  sanitārs_izgāztuve
briedis  ---  briedis
statistikas dati  ---  statistika_dati
Haga slimība  ---  hags_slimība
slimība  ---  slimība
endēmiskās sugas  ---  endēmisks_sugas
ziedaugi  ---  ziedaugs
ieleja  ---  ieleja
notekūdeņi  ---  notekūdens
=== EXCLUDE  radioaktīvo treiseru tehnika
mīts  ---  mīt
=== EXCLUDE  alkila savienojums
diferencēšana; atšķiršana; sadalīšana  ---  diferencēšana_atšķiršana_sadalīšana
stabils  ---  stabils
krāsns  ---  krāsns
tiesības saņemt kompensāciju  ---  tiesības_saņemt_kompensācija
nevalstiska organizācija  ---  nevalstisks_organizācija
kalcija saturs  ---  kalcijs_saturs
pludmaļu tīrīšana  ---  pludmale_tīrīšana
riska samazināšana  ---  riska_samazināšana
šķidrais kurināmais mājsaimniecības apsildei  ---  šķidrais_kuri

apvienība; asociācija  ---  apvienība_asociācija
alumīnija sastāvs  ---  alumīnija_sastāvs
tēze  ---  tēze
municipalitātes veiktie sanitārijas pasākumi  ---  municipalitāte_veikt_sanitārija_pasākums
broms  ---  broms
dabiska izplešanās  ---  dabisks_izplešanās
lineārs (piesārņojuma) avots  ---  lineārs_piesārņojums_avots
gumijas atkritumi  ---  gumija_atkritumi
aizstājamība (ķīmija)  ---  aizstājamība_ķīmija
sankcija  ---  sankcija
krāsainais metāls  ---  krāsains_metāls
joslveida derīgo izrakteņu ieguve  ---  joslveida_derīgs_izraktenis_ieguve
skaņas pārraide  ---  skaņa_pārraide
pakalns; uzkalns  ---  pakalns_uzkalns
spēles (spēle)  ---  spēle_spēle
atkritumu izmantošana termoelektrostaciju darbībā  ---  atkritumi_izmantošana_termoelektrostacija_darbība
kodolsintēze  ---  kodolsintēze
traucējums  ---  traucējums
atkritumu klasificēšana  ---  atkritumi_klasificēšana
vienošanās par atdodamā izmantotā iesaiņojuma depozītu  ---  vienošanās_atdodams_izmantot_iesaiņojums_depozīts
daļiņa  -

pārvalde  ---  pārvalde
sadedzināšanas atlikumi  ---  sadedzināšana_atlikums
emulsifikācija  ---  emulsifikācija
pret piesārņojuma stimulēšana  ---  piesārņojums_stimulēšans
freons  ---  freons
sastāva kvalitatīva un kvantitatīva noteikšana  ---  sastaut_kvalitatīvs_kvantitatīvs_noteikšana
dokuments  ---  dokuments
atpūtas nometne  ---  atpūt_nometne
nometne  ---  nometne
alergēni  ---  alergēns
ērce  ---  ērce
tirdzniecības šķērslis  ---  tirdzniecība_šķērslis
pedosfēra  ---  pedosfēra
sālsskābe  ---  sālsskābe
atkritumu izgāztuve  ---  atkritumi_izgāztuve
zāģu skaidas  ---  zāģis_skaida
krīts  ---  krīt
stroncijs  ---  stroncijs
termodinamika  ---  termodinamika
likumpārkāpums  ---  likumpārkāpums
zirgs  ---  zirgs
šaušanai paredzētā vieta  ---  šaušana_paredzēt_vieta
karjers  ---  karjers
karjeru izstrāde  ---  karjera_izstrāde
viadukts; gaisa tilts  ---  viadukt_gaiss_tilts
sakne  ---  sakne
apvienošanās  ---  apvienošanās
parametrs  ---  parametrs
hlorīds  ---  hlorīds
balsošana  

=== EXCLUDE  vairākreizējas lietošanas konteiners
sabiedriskās domas aptauja  ---  sabiedrisks_doma_aptauja
parazitoloģija  ---  parazitoloģija
=== EXCLUDE  triazīns
punkts  ---  punkts
līme  ---  līme
prejudiciālā tiesvedība  ---  prejudiciāls_tiesvedība
=== EXCLUDE  skaņas imisija
putnkopība  ---  putnkopība
nomenklatūra  ---  nomenklatūra
mikroelements  ---  mikroelements
anomālija; izkropļojums  ---  anomālijs_izkropļojums
atkritumu izgāztuve  ---  atkritumi_izgāztuve
sakausējums  ---  sakausējums
aizstāvība; aizsardzība  ---  aizstāvība_aizsardzība
kvēpi  ---  kvēps
kristalogrāfija  ---  kristalogrāfija
plutonijs  ---  plutonijs
=== EXCLUDE  ksenobiotiska viela
izdevīgs karogs  ---  izdevīgs_karogs
imunitāte  ---  imunitāte
baktericīds  ---  baktericīds
nekroze  ---  nekroze
pārveidotu mikroorganismu atbrīvošanās (nonākšana vidē)  ---  pārveidot_mikroorganisms_atbrīvošana_nonākšana_vide
ēteris  ---  ēteris
līcis  ---  līcis
petroleja  ---  petroleja
līdzsadedzināšana  ---  līdzsad

=== EXCLUDE  mikropiesārņotājs
raktuve; šahta  ---  raktuve_šahta
=== EXCLUDE  pārizmantošana
pārmēslošana  ---  pārmēslošana
=== EXCLUDE  peroksiacetilnitrāts
=== EXCLUDE  fitomasa
roņveidīgais  ---  roņveidīgs
pēdminis  ---  pēdminis
=== EXCLUDE  polihlordibenzofurāns
autekoloģija  ---  autekoloģija
snuķaiņu suga  ---  snuķaiņa_suga
agrotūrisms  ---  agrotūrisms
skate  ---  skate
sniega lavīna  ---  sniegs_lavīns
sūklis  ---  sūklis
krājumu uzskaite  ---  krājums_uzskaite
bioētika  ---  bioētika
=== EXCLUDE  traheofīts
=== EXCLUDE  UNCED
lāču dzimta  ---  lācis_dzimt
bioreaktors  ---  bioreaktors
bioloģiskā drošība  ---  bioloģisks_drošība
aka  ---  aka
grāmatvedība  ---  grāmatvedība
Excluded:  183
Final topics:  5125 5125


In [30]:
# adding all the words and topics to model

# load the lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()
# load the nolemma list
with open(nolemma_list_file_excluded) as fp:
    topics_nolemma = fp.readlines()    
    

#try if the topic-lemma words exist in the model     
new_vectors = []
new_words = []
for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
#for i,topic in enumerate(topics):
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text) 
            
    topic = topic.strip()
    topic_words = topic.split('_')
    
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    #if len(topic_words)<2:
    #    continue
    
    
    tmp_vectors = []
    
    for word, word_nolemma in zip(topic_words, nolemmas):
    #for word in topic_words:
        vector = get_vector(word)
        
        try:
            tmp_vectors.append(model[word])
        except: 
            print('\nNOT in vocabulary: ', word, topic)
            vector = get_vector(word)
            if vector != []:
                tmp_vectors.append(vector)
                if word not in new_words:
                    print('append ', word)
                    new_vectors.append(vector)
                    new_words.append(word)   
            else:
                vector = get_vector(word_nolemma)
                if vector != []:
                    tmp_vectors.append(vector)
                    if word not in new_words:
                        print('append ', word)
                        new_vectors.append(vector)
                        new_words.append(word)  
                else:
                    raise Warning(f"XXX {word} {topic} {topic_nolemma}")
                    print("XXX", word, topic, topic_nolemma) 
    #if word not in new_words:
    #    new_vectors.append(list(np.average(tmp_vectors, axis=0)))
    #    new_words.append(word)        
    if len(tmp_vectors) > 1:
        if topic not in new_words:
   
            print('append ', topic)
            new_vectors.append(list(np.average(tmp_vectors, axis=0)))
            new_words.append(topic)
        else:
            print('doubled TOPIC: ', topic)    
    else:
        print('--- only one word, not a new topic>', topic)

append  vietējs_attīstība
append  lauks_attīstība
append  ūdens_resursi_apsaimniekošana
append  izglītība_līmenis
append  vispārējs_izglītība
append  banka_zeme
append  kopiena_tiesības
append  kultūra_mantojums
append  lauksaimniecības_zeme
append  lauksaimniecības_ainava
append  lauksaimniecisks_ražošana
append  ekonomisks_attīstība
append  barība_ķēde
append  pārtika_rūpniecība
append  pārtika_kvalitāte
append  globāls_sasilšana
append  siltumnīca_efekts
append  rūpniecisks_attīstība
append  darbs_tirgus
append  pieeja_zeme
append  zeme_gabals_apvienošana
append  zeme_plānošana
append  ainava_arhitektūra
append  ainava_plānošana
append  zeme_lietošana
append  zeme_lietošana_plānošana
append  jūra_vide
append  daba_aizsardzība
doubled TOPIC:  daba_aizsardzība
append  bioloģisks_lauksaimniecībs
append  fizikāls_plānošana

NOT in vocabulary:  sākumizglītība sākumizglītība
append  sākumizglītība
--- only one word, not a new topic> sākumizglītība
--- only one word, not a new topic> autor

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


append  augsne_paskābināšanās
append  bioloģisks_mantojums
append  kopiena_akts
append  ferma_ēka
append  rūpniecisks_ražošana_ēka
--- only one word, not a new topic> klimats
append  kosmētika_rūpniecība
--- only one word, not a new topic> lopkopība
append  resursi_izlietošana
append  ūdens_izmantošana
append  zālājs_pļava

NOT in vocabulary:  tēraudīt tēraudīt_rūpniecība
append  tēraudīt
append  tēraudīt_rūpniecība
append  elektrotehnisks_rūpniecība
append  ūdensšķirtne_pārvaldība
append  ekosistēma_degradācija
append  apģērbs_ražošana
append  keramika_rūpniecība
append  tirgus_ekonomika
append  vide_pārvaldība
append  programmatūra_attīstība
append  lauksaimniecības_produkts_uzglabāšana
--- only one word, not a new topic> zemkopība
append  lauksaimniecības_izstrādājums
append  klimats_izmainīšanās
append  lauksaimniecības_ekonomika
append  zeme_attīstība
append  rūpniecība_plānošana
append  ekonomika_plānošana
--- only one word, not a new topic> arhitektūra
append  pārtika_tirdzniecī


NOT in vocabulary:  iesāļš ciets_ūdens_iesāļš_ūdens
append  iesāļš
append  ciets_ūdens_iesāļš_ūdens
append  augs_mantojums
append  pilsēta_satiksme
append  augsts_precizitāte_inženiertehnoloģija

NOT in vocabulary:  sārmainīt sārmainīt_augsne
append  sārmainīt
append  sārmainīt_augsne
append  es_padome

NOT in vocabulary:  urbānāties urbānāties_struktūra
append  urbānāties
append  urbānāties_struktūra
append  ainava_atjaunošana
append  radiācija_ietekme
append  ūdens_organisms
append  saldūdens_organisms
append  jūra_organisms
append  vide_uzraudzība
append  celuloze_rūpniecība
--- only one word, not a new topic> siltumsūknis
--- only one word, not a new topic> dzīvesveids
append  enerģija_pārveide_enerģija_pārveidošanās
append  vadība_plāns
--- only one word, not a new topic> minerālūdens
doubled TOPIC:  celuloze_rūpniecība
append  valsts_finanses
append  zeme_iecirknis_līdzināšana_profilēšana
append  ainava_sastāvdaļa
append  tiesības_sistēma
append  jūtīgs_teritorija
append  likums

append  arktika_reģions
append  vēsturisks_evolūcija
append  vide_draudzīgs_ieguldījums
append  vide_prioritāte
append  vide_dati
append  stihisks_nelaime_daba_katastrofa
append  ekoloģiskais_stimuls
append  urbāns_vide_apdzīvot_vieta
append  plānošana_pasākums
append  dzīvnieks_uzvedība
append  tirgus_izpēte
append  komerciāls_dārzkopība
append  augs_populācija

NOT in vocabulary:  demineralizācija ūdens_demineralizācija
append  demineralizācija
append  ūdens_demineralizācija

NOT in vocabulary:  ieviešanīt likums_ieviešanīt_tiesības_norma_ieviešana
append  ieviešanīt
append  likums_ieviešanīt_tiesības_norma_ieviešana
append  jaunietis_darbs
append  izpilde_likums
append  bažas_vide
append  valsts_iestāde
append  akmeņoglis_rūpniecība
append  rūpniecisks_notekūdens_dūņas
--- only one word, not a new topic> sabiedrība
append  sociāls_analīze
append  kultūra_centrs
--- only one word, not a new topic> reģions
append  izmeši_ūdens
append  ekonomisks_plāns
append  civils_drošība
append  au

...

append  laboratorijas_eksperiments
append  bioķīmisks_viela
append  muita_nodot
--- only one word, not a new topic> attīrīšana
--- only one word, not a new topic> akcīze
append  vērtēšana_kritērijs
append  augs_suga_ieviesšana
--- only one word, not a new topic> cietība
--- only one word, not a new topic> caurule
append  dokuments_tips
append  valsts_pārvalde_forma
append  baterija_iznīcināšana
--- only one word, not a new topic> nogulsne
--- only one word, not a new topic> golfs

NOT in vocabulary:  amonifikācija amonifikācija
append  amonifikācija
--- only one word, not a new topic> amonifikācija
append  ievešana_atļauja
doubled TOPIC:  tirdzniecība_ierobežojums
--- only one word, not a new topic> autostāvvieta
--- only one word, not a new topic> monopols
--- only one word, not a new topic> dūmi
--- only one word, not a new topic> zaudējums
--- only one word, not a new topic> kempings
--- only one word, not a new topic> hidrobioloģija
append  gliemene_austere
--- only one word, not a


NOT in vocabulary:  asean asean_dienvidaustrumāzija_valsts_asociācija
append  asean
append  asean_dienvidaustrumāzija_valsts_asociācija
append  radioaktīvs_viela
--- only one word, not a new topic> pašvaldība
append  brīvs_tirdzniecība
--- only one word, not a new topic> ogle
--- only one word, not a new topic> iztvaikošana
append  izlase_veids_pārbaude
--- only one word, not a new topic> tilts
append  ogle_kurināt_elektrostacija
doubled TOPIC:  kontrolēt_dedzināšana
--- only one word, not a new topic> pārbaude
--- only one word, not a new topic> dīķis
--- only one word, not a new topic> aktinīds
append  ziņojums_ministrs
append  lielceļš_automaģistrāle
append  sāls_korķis
--- only one word, not a new topic> pārvalde
append  sadedzināšana_atlikums

NOT in vocabulary:  emulsifikācija emulsifikācija
append  emulsifikācija
--- only one word, not a new topic> emulsifikācija
append  piesārņojums_stimulēšans
--- only one word, not a new topic> freons

NOT in vocabulary:  sastaut sastaut_kva

append  tradīcija_izcirst_izdedzināt_mežs_palielināt_lauksaimniecība_platība
--- only one word, not a new topic> apskate

NOT in vocabulary:  halogenēt halogenēt_terfenils

NOT in vocabulary:  terfenils halogenēt_terfenils
append  terfenils
append  halogenēt_terfenils
--- only one word, not a new topic> amalgams

NOT in vocabulary:  saprobitāte saprobitāte_indekss
append  saprobitāte
append  saprobitāte_indekss
append  dati_aprēķināt_viens_iedzīvotājs
--- only one word, not a new topic> reģistrācija
append  noteikums_atcelšana_ierobežojums_atcelšana
append  dzesēt_viela
--- only one word, not a new topic> rezervāts
append  hordainis_dzīvnieks_tips_hordainis
append  navigācija_briesmas
append  risks_briesmas_apdraudējums
--- only one word, not a new topic> saneši
--- only one word, not a new topic> malārija
append  slīpēšana_atkritumi
--- only one word, not a new topic> smēķēšans
doubled TOPIC:  augsne_zinātne
--- only one word, not a new topic> svars

NOT in vocabulary:  šķiedenis šķie

--- only one word, not a new topic> ēteris
--- only one word, not a new topic> līcis
--- only one word, not a new topic> petroleja

NOT in vocabulary:  līdzsadedzināšana līdzsadedzināšana
append  līdzsadedzināšana
--- only one word, not a new topic> līdzsadedzināšana
--- only one word, not a new topic> delta

NOT in vocabulary:  dozējums dozēšana_dozējums
append  dozējums
append  dozēšana_dozējums
append  atkritumi_šķirošana_tā_veidošanās_vieta
append  mācība_darbs
--- only one word, not a new topic> samazināšana
--- only one word, not a new topic> emulsija
append  pārāk_augsts_skurstenis
--- only one word, not a new topic> ontoģenēze
--- only one word, not a new topic> indekss
append  dzirde_asums
append  pārgājiens_taka

NOT in vocabulary:  iedīglt patogēns_mikroorganisms_iedīglt_aizmetnis
append  iedīglt
append  patogēns_mikroorganisms_iedīglt_aizmetnis
--- only one word, not a new topic> prusaks
doubled TOPIC:  atkritumi_izgāšana
--- only one word, not a new topic> aizsprost
append

append  masas
append  ledus_slānis_masas_bilance
append  zeme_degradācija
append  atkritumi_poligons_direktīva
append  kuģis_degviels
append  jūra_stratēģija_pamatdirektīva
append  sadzīve_atkritumi_rašanās
append  komunāls_notekūdens
append  nitrāts_direktīva
append  organizācija_vide_pēda_nospiedums_metode
append  ozona_noārdīšanās
append  parks_atkritumi
append  politika_sistēma
append  politika_īstenošana
append  primārs_enerģija
append  produkts_vide_pēda_nospiedums_metode
append  patērētājs_ražot_produkcija
append  direktīva_atjaunot_energoresurss_izmantošana_veicināšana
append  resursi_produktivitāte
append  upe_baseins
append  ceļš_satiksme_troksnis
append  ilgtspējīgs_attīstība_mērķis
append  ilgtspējīgs_mežs_apsaimniekošana
append  pāreja_ūdens
append  ūdens_pamatdirektīva
doubled TOPIC:  mājturība_atkritumi
append  iekštelpa_dūmi
append  i_inspire_i
append  poligons_atkritumi
append  poligons_atkritumi_plūsma
append  organisks_notekūdens
append  politika_efektivitāte
append 

In [31]:
# add topics to model
model.add(new_words,new_vectors)

In [32]:
# save the model with new topics
model.save_word2vec_format(excluded_model_name, binary=True)
#reload the model  otherwise cannot find similarities fo newly added words
model = KeyedVectors.load_word2vec_format(excluded_model_name, binary=True)

In [33]:
len(model.index2word)

126658

In [36]:
# try the model
model.similar_by_vector('lauks_attīstība')

[('lauks_teritorija', 0.9055083990097046),
 ('lauks', 0.9042763710021973),
 ('lauks_saimniecība', 0.8971288204193115),
 ('lauks_iedzīvotājs', 0.887458086013794),
 ('lauks_arhitektūra', 0.8719847202301025),
 ('lauks_teritorija_pārvaldība_plānošana', 0.8425607681274414),
 ('ūdensapgāde_lauks_teritorija', 0.8405729532241821),
 ('lauks_vide', 0.8350191712379456),
 ('likums_lauks_teritorija', 0.8115594387054443),
 ('lauks_dzīvot_vide', 0.8107056617736816)]

In [37]:
# find most similar to topics to terms in the new model

# load the new lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()

for i, topic in enumerate(topics):
    topic = topic.strip()
    print('\n-----\n', topic)
    try:
        print(i, model.similar_by_vector(topic))
    except:
        print(topic, 'NOT in model')


-----
 vietējs_attīstība
0 [('vietējs_pašvaldība_politika', 0.847332239151001), ('vietējs', 0.8437862396240234), ('reģionāls_attīstība', 0.8191958665847778), ('vietējs_finanses', 0.7981908321380615), ('vietējs_celtniecība_materiāls', 0.783234715461731), ('vietējs_apmežošana', 0.7789285778999329), ('ekonomisks_attīstība', 0.7686658501625061), ('pilsēta_attīstība', 0.7651100158691406), ('vide_vietējs_tehnoloģija', 0.7651031613349915), ('vietējs_apstāklis_pārzināšana_zināšana_balstīt_vietējs_apstāklis_izpēt', 0.759191632270813)]

-----
 lauks_attīstība
1 [('lauks_teritorija', 0.9055083990097046), ('lauks', 0.9042763710021973), ('lauks_saimniecība', 0.8971288204193115), ('lauks_iedzīvotājs', 0.887458086013794), ('lauks_arhitektūra', 0.8719847202301025), ('lauks_teritorija_pārvaldība_plānošana', 0.8425607681274414), ('ūdensapgāde_lauks_teritorija', 0.8405729532241821), ('lauks_vide', 0.8350191712379456), ('likums_lauks_teritorija', 0.8115594387054443), ('lauks_dzīvot_vide', 0.8107056617736

30 [('fizikāls', 0.8638843297958374), ('fizikāls_process', 0.854561448097229), ('fizikāls_piesārņojums', 0.8297981023788452), ('fizikāls_apstāklis', 0.8295619487762451), ('fizikāls_ģeogrāfija', 0.8216845989227295), ('fizikāls_zinātne', 0.8215398788452148), ('fizikāls_apstrāde', 0.8208870887756348), ('fizikāls_izmaiņas', 0.8144721984863281), ('fizikāls_okeanogrāfija', 0.8066611886024475), ('fizikāls_piesārņotājs', 0.8000145554542542)]

-----
 sākumizglītība
31 [('izglītība', 0.6555471420288086), ('vērtībizglītība', 0.6528742909431458), ('papildizglītība', 0.6508280634880066), ('pašizglītība', 0.6275626420974731), ('pieaugt_izglītība', 0.627302885055542), ('arodizglītība', 0.6256670355796814), ('vispārējs_izglītība', 0.6223751306533813), ('izglītošana_izglītība_mācība', 0.6179184913635254), ('sākums', 0.6134510040283203), ('izglītība_plānošana', 0.6124809980392456)]

-----
 autorūpniecība
32 [('autorūpnīca', 0.7638999223709106), ('aviorūpniecība', 0.7186208367347717), ('transportlīdzekli

59 [('zeme_plānošana', 0.963228702545166), ('zeme_lietošana_plānošana', 0.9316937923431396), ('zeme_plānošana_reģionāls_plānošana', 0.9284693598747253), ('zeme_attīstība', 0.8715558052062988), ('zeme_izmantošana', 0.8517061471939087), ('lauks_teritorija_pārvaldība_plānošana', 0.8508473634719849), ('lauksaimniecības_zeme_apsaimniekošana', 0.8494832515716553), ('zeme_saglabāšana', 0.8386384844779968), ('zeme_atjaunošana', 0.8385541439056396), ('zeme_piešķiršana', 0.8233902454376221)]

-----
 ūdens_resursi_attīstība
60 [('ūdens_resursi', 0.9504029750823975), ('ūdens_resursi_apsaimniekošana', 0.9457552433013916), ('ūdens_resursi_saglabāšana', 0.9327445030212402), ('ūdens_resursi_attiecināt_likumdošana', 0.876526951789856), ('ūdens_izmantošana', 0.8590370416641235), ('ūdens_lauksaimniecības_vajadzība', 0.8466113805770874), ('ūdens_aizsardzība_teritorija', 0.8442932367324829), ('ūdens_atjaunošana', 0.8422932624816895), ('ūdens_dzinējspēks_ūdens_enerģija', 0.8419756889343262), ('ūdens_daudzum

88 [('daba_resursi_pārvaldība', 0.95615553855896), ('daba_resursi_saglabāšana', 0.950533390045166), ('daba_resursi_degradācija', 0.924511730670929), ('daba', 0.8957688808441162), ('daba_aizsardzība', 0.8906649351119995), ('daba_aizsardzība_organizācija', 0.876544713973999), ('daba_aizsardzība_programma', 0.8750858902931213), ('daba_aizsardzība_politika', 0.8644446134567261), ('likumdošana_daba_aizsardzība', 0.8532100915908813), ('daba_saglabāšana_politika', 0.8503923416137695)]

-----
 mežs_resursi
89 [('mežs_resursi_novērtējums', 0.9434763193130493), ('mežs_izmantošana', 0.9138805866241455), ('mežs_resursi_iznīkšana', 0.9044833183288574), ('mežs_aizsardzība', 0.9006354212760925), ('mežs', 0.8994379639625549), ('mežs_ekonomika', 0.8930615186691284), ('mežs_politika', 0.8901557922363281), ('ilgtspējīgs_mežs_apsaimniekošana', 0.8866714835166931), ('mežs_atjaunošana', 0.8829442262649536), ('mežs_saglabāšana', 0.8770390748977661)]

-----
 atjaunot_resursi
90 [('neatjaunojams_resursi', 0.86

111 [('ūdens', 0.9192169904708862), ('ūdens_demineralizācija', 0.9035409092903137), ('ūdens_aerācija', 0.9004004001617432), ('ūdens_apsaimniekošana', 0.8916837573051453), ('ūdens_piesārņojums', 0.8914027810096741), ('ūdens_attīrīšana', 0.8832020163536072), ('ūdens_izmantošana', 0.8814516067504883), ('ūdens_aizsardzība', 0.8774980306625366), ('ūdens_piesārņotājs', 0.866268515586853), ('ūdens_transportēšana', 0.8604179620742798)]

-----
 augsne_iegrimšana
112 [('augsne_izmantošana', 0.8728188276290894), ('augsne_stabilizēšanās', 0.8598791360855103), ('augsne', 0.8556836843490601), ('augsne_sablīvēšanās', 0.8553918600082397), ('augsne_paskābināšanās', 0.8500523567199707), ('augsne_izskalošanās', 0.8494201898574829), ('augsne_stabilitāte', 0.8476561307907104), ('augsne_degradēšanās', 0.8433171510696411), ('augsne_apstrādāšana', 0.8428826332092285), ('augsne_saglabāšana', 0.8403752446174622)]

-----
 demogrāfisks_attīstība
113 [('ekonomisks_attīstība', 0.8711622953414917), ('demogrāfisks', 

...

4982 [('pretalkohols', 0.5485541820526123), ('gliftāls', 0.4475365877151489), ('pienas', 0.44526249170303345), ('autodegviela', 0.4440332353115082), ('bezkofeīns', 0.44306445121765137), ('autogēns', 0.44065552949905396), ('bezsvins', 0.44065433740615845), ('guarāns', 0.4332113265991211), ('autodurvis', 0.4322127401828766), ('galantamīns', 0.4298548102378845)]

-----
 pārstrāde
4983 [('atkritumi_pārstrāde', 0.8868725895881653), ('atkritumi_pārstrāde_tehnoloģija', 0.8716263771057129), ('atkritumi_pārstrāde_otrreizējs_izmantošana', 0.8656494617462158), ('atkritumi_pārstrāde_rūpniecība', 0.8615844249725342), ('pārpalikums_atkritumi_pārstrāde', 0.8485704064369202), ('atkritumi_pārstrāde_rūpnīca', 0.8345712423324585), ('notekūdens_pārstrāde', 0.828453540802002), ('pārstrāde_atlikums', 0.8024086952209473), ('individuāls_notekūdens_pārstrāde', 0.789336085319519), ('notekūdens_pārstrāde_sistēma', 0.7826415300369263)]

-----
 sārms
4984 [('sārma', 0.8550529479980469), ('skābe', 0.753621518611908

5012 [('ekoloģisks_kapacitāte', 0.8212027549743652), ('fotoķīmisks_produkts', 0.7922681570053101), ('pārtika_produkts_apstarošana', 0.7804652452468872), ('bioloģisks_aktivitāte', 0.7737667560577393), ('produkts', 0.7710041999816895), ('lauksaimniecības_produkts_uzglabāšana', 0.7685827016830444), ('apstrādāt_pārtika_produkts', 0.7662997245788574), ('pārtika_produkts_transportēšana', 0.7609249949455261), ('pārtika_produkts_apstrāde_nozare', 0.7590194940567017), ('bioloģisks_resursi', 0.7590154409408569)]

-----
 ķīmisks_komerciāls_atkritumi
5013 [('ķīmisks_atkritumi', 0.9423742890357971), ('atkritumi_ķīmisks_apstrāde', 0.8931472301483154), ('ķīmisks', 0.8779243230819702), ('ķīmisks_piesārņojums', 0.8775718212127686), ('ķīmisks_izstrādājums', 0.8606989979743958), ('ķīmisks_piesārņotājs', 0.8568850755691528), ('ķīmisks_piemaisījums', 0.8329451084136963), ('speciāls_ķīmisks_izstrādājums', 0.8325855731964111), ('ķīmisks_savienojums', 0.8302654027938843), ('ķīmisks_attīrīšana', 0.826241850852

5043 [('ex', 0.9179737567901611), ('de', 0.6271619200706482), ('v', 0.6262523531913757), ('is', 0.6204110383987427), ('pārbaude_in_vivot', 0.617324709892273), ('se', 0.6144107580184937), ('in', 0.6140350103378296), ('hs', 0.6023848652839661), ('des', 0.6002355813980103), ('ms', 0.5998049378395081)]

-----
 mēslošana_līdzeklis_patēriņš
5044 [('likums_mēslošana_līdzeklis', 0.8323102593421936), ('mēslošana', 0.7983203530311584), ('tirdzniecība_patēriņš', 0.7523654699325562), ('patēriņš', 0.7460353970527649), ('patēriņš_produkts', 0.7423886060714722), ('dezinfekcija_līdzeklis', 0.741586446762085), ('augsne_mēslošana', 0.7245744466781616), ('līdzeklis', 0.7200252413749695), ('augs_aizsardzība_līdzeklis', 0.7198717594146729), ('izejviela_patēriņš', 0.7197892069816589)]

-----
 iespējams_plūdi_postījums
5045 [('plūdi', 0.9138047099113464), ('aizsardzība_plūdi', 0.9097846746444702), ('plūdi_applūšana', 0.8859884142875671), ('plūdi_prognoze', 0.8627066612243652), ('plūdi_novēršans', 0.859331965

5074 [('īstenošana', 0.8677726984024048), ('attīstība_politika', 0.8586462736129761), ('politika_plānošana', 0.8457109928131104), ('politika', 0.8336279392242432), ('politika_efektivitāte', 0.8186742663383484), ('izglītība_politika', 0.8134877681732178), ('politika_pamatnostādne', 0.8004708886146545), ('politika_integrēšana', 0.7977871894836426), ('tirdzniecība_politika', 0.7963542938232422), ('komunikācija_politika', 0.7941019535064697)]

-----
 primārs_enerģija
5075 [('primārs_enerģija_patēriņš', 0.9162112474441528), ('primārs', 0.8109285831451416), ('tradicionāls_enerģija', 0.8040760159492493), ('netradicionāls_enerģija', 0.8037664294242859), ('enerģija', 0.7940893173217773), ('enerģija_veids', 0.771087646484375), ('enerģija_disipācija', 0.756293535232544), ('atjaunojat_enerģija_resursi', 0.7559728026390076), ('pieprasījums_enerģija', 0.7532011270523071), ('enerģija_saglabāšana', 0.7429854273796082)]

-----
 produkts_vide_pēda_nospiedums_metode
5076 [('organizācija_vide_pēda_nospied

5102 [('gāze_uzņēmums', 0.8558712005615234), ('gāze_elektrostacija', 0.8540792465209961), ('gāze', 0.8453004360198975), ('sašķidrināt_gāze', 0.8012996912002563), ('gāze_hromatogrāfija', 0.8011318445205688), ('gāze_saimniecība_gāzapgāde', 0.7959070205688477), ('atkritumi_gāze', 0.7953575849533081), ('gāze_attīrīšana', 0.7879826426506042), ('transportlīdzeklis_izplūde_gāze', 0.7853052616119385), ('izplūde_gāze', 0.7791464924812317)]

-----
 zaļš_josla
5103 [('klimatisks_josla', 0.837527871131897), ('josla', 0.8369892239570618), ('mērenas_klimatisks_josla_mežs', 0.8007833957672119), ('spektrāls_josla', 0.7973670959472656), ('zaļš_zona', 0.7855187058448792), ('pilsēta_zaļš_zona', 0.7666932344436646), ('krasts_līnija_paralelt_smilts_josla_jūra', 0.762512743473053), ('zaļš', 0.7499639987945557), ('zaļš_koridors', 0.7437740564346313), ('zaļš_paisums', 0.7136131525039673)]

-----
 haloforma_haloforma_reakcija
5104 [('haloforma', 0.9402556419372559), ('reakcija', 0.7510956525802612), ('fotoķīmi